In [1]:
import regex
import numpy as np
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
import spacy
import pyLDAvis

/Users/evandimartinis/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/Users/evandimartinis/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/Users/evandimartinis/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_mo

In [2]:
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

/Users/evandimartinis/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/image.py:172: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/Users/evandimartinis/opt/anaconda3/lib/python3.8/site-packages/scipy/io/matlab/mio5.py:95: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and 

In [3]:
df = pd.read_csv('CleanedNews.csv')
df.head(10)

,Unnamed: 0,Unnamed: 0.1,title,score,id,url,coms_num,author,upvotes,Unnamed: 0.1.1,Unnamed: 0.1.1.1,word_count
0,0,0,university michigan officials didnt act report...,87,nawgp1,https://www.cnn.com/2021/05/12/us/umichigan-in...,9,JRWoodwardMSW,87,NaN,NaN,19
1,1,1,police west bath boy 2 injures parents self sh...,0,naw8ps,https://www.centralmaine.com/2021/05/12/police...,12,fbreaker,0,NaN,NaN,10
2,2,2,virginia stations run gas pipeline shutdown co...,185,na1ooq,https://www.nbc12.com/2021/05/10/some-virginia...,83,Sure-Ad7730,185,NaN,NaN,11
3,3,3,john mcafee close bankruptcy fights crypto fra...,360,nab0wk,https://www.cityam.com/john-mcafee-close-to-ba...,66,EVERWILDOUTDOORS,360,NaN,NaN,11
4,4,4,met officer stole rival gangs drugs cash fake ...,35,nawn6z,https://www.theguardian.com/uk-news/2021/may/1...,4,Hillick,35,NaN,NaN,11
5,5,5,disney pay star wars novelists alan dean foster,1537,nac2qk,https://bleedingcool.com/comics/disney-pay-sta...,118,boringhistoryfan,1537,NaN,NaN,11
6,6,6,hackers paralyzed pipeline banks stock exchang...,33,naxifv,https://edition.cnn.com/2021/05/12/business/ra...,4,WebLinkr,33,NaN,NaN,12
7,7,7,times square shooting suspect caught florida,14,naxpqn,https://www.nbcnews.com/news/us-news/times-squ...,0,zsalzman12,14,NaN,NaN,7
8,8,8,us envoy heading middle east urge calm israel ...,42,nau5g8,https://www.nytimes.com/2021/05/12/world/middl...,42,Hillick,42,NaN,NaN,16
9,9,9,28yearold woman posed student sneaked high sch...,46,nax5vs,https://www.nbcnews.com/news/us-news/28-year-o...,32,Hillick,46,NaN,NaN,13


In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data = df['title'].tolist()
data_words = list(sent_to_words(data))

print(data_words[0])

['university', 'michigan', 'officials', 'didnt', 'act', 'reports', 'sexual', 'misconduct', 'former', 'university', 'physician', 'decades', 'report', 'says']


In [5]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
bigram_mod = gensim.models.phrases.Phraser(bigram)
print(bigram_mod[data_words[0]])

['university', 'michigan', 'officials', 'didnt', 'act', 'reports', 'sexual', 'misconduct', 'former', 'university', 'physician', 'decades', 'report', 'says']


In [6]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

['university', 'michigan', 'official', 'do', 'act', 'report', 'sexual', 'misconduct', 'former', 'university', 'physician', 'decade', 'report', 'say']


In [7]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 2)]


In [18]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
# Print the KeywordS in the 20 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.015*"police" + 0.014*"gaza" + 0.012*"fuel" + 0.011*"woman" + 0.010*"israel" + 0.009*"attack" + 0.009*"rocket" + 0.009*"officer" + 0.009*"death" + 0.009*"pipeline"'), (1, '0.021*"covid" + 0.016*"say" + 0.011*"vaccine" + 0.009*"hama" + 0.009*"group" + 0.009*"rule" + 0.009*"right" + 0.008*"first" + 0.008*"federal" + 0.008*"kill"'), (2, '0.016*"covid" + 0.015*"man" + 0.015*"arrest" + 0.014*"new" + 0.009*"pipeline" + 0.009*"county" + 0.009*"girl" + 0.009*"month" + 0.008*"report" + 0.008*"record"'), (3, '0.018*"pay" + 0.015*"gas" + 0.013*"pipeline" + 0.012*"charge" + 0.011*"school" + 0.008*"sentence" + 0.008*"murder" + 0.008*"death" + 0.007*"police" + 0.007*"indict"')]


In [19]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds')
vis

/Users/evandimartinis/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:57: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/Users/evandimartinis/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:57: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/Users/evandimartinis/

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.181221 -0.121691       1        1  33.567022
2     -0.108261  0.168731       2        1  22.897772
0      0.124274 -0.166968       3        1  22.709993
1      0.165208  0.119928       4        1  20.825213, topic_info=        Term       Freq      Total Category  logprob  loglift
49       pay  20.000000  20.000000  Default  30.0000  30.0000
242   arrest  11.000000  11.000000  Default  29.0000  29.0000
21       gas  16.000000  16.000000  Default  28.0000  28.0000
210     gaza  10.000000  10.000000  Default  27.0000  27.0000
208  vaccine   8.000000   8.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
232     last   4.202320   8.844624   Topic4  -5.0903   0.8248
130    judge   4.611194  12.943189   Topic4  -4.9975   0.5369
132      man   4.661988  22.234436   Topic4  -4.9865   0.0068
172    price   3.614614   7.364591   Topic4  -5.2410   0.8573
223     rise   3.435791   5.913586   Topic4  -5.2917   1.0260

[210 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
186       1  0.375676   accuse
186       3  0.626126   accuse
482       1  0.849045    actor
203       4  0.866135      age
221       3  0.761255      air
...     ...       ...      ...
507       4  0.228289   worker
150       1  0.226319     year
150       2  0.565797     year
150       4  0.226319     year
146       2  0.959066  yearold

[218 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 1, 2])